In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import sys, time, datetime, shutil, os
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
import csv
import cv2
import matplotlib.pyplot as plt

In [ ]:
# Global variables
BATCH_SIZE = 32
NB_EPOCHS = 100
IMG_WIDTH = 96
IMG_HEIGHT = 96
TRAIN_DATA_PATH = 'chest_xray/train'
TEST_DATA_PATH = 'chest_xray/test'
VAL_DATA_PATH = 'chest_xray/val'
CLASS_NAMES = ['NORMAL', 'BACTERIA', 'VIRUS']
NUM_CLASSES = 3
AUTOTUNE = tf.data.experimental.AUTOTUNE
VERBOSE = 1
OPTIMIZER = 'adam'
HIDDEN_ACTIVATION = 'relu'
FINAL_ACTIVATION = 'sigmoid'

METRICS = [
  tf.keras.metrics.CategoricalAccuracy(name='accuracy', dtype=tf.float32),
  tf.keras.metrics.TruePositives(name='true_positives', dtype=tf.float32),
  tf.keras.metrics.FalsePositives(name='false_positives', dtype=tf.float32),
  tf.keras.metrics.TrueNegatives(name='true_negatives', dtype=tf.float32),
  tf.keras.metrics.FalseNegatives(name='false_negatives', dtype=tf.float32), 
  tf.keras.metrics.Precision(name='precision', dtype=tf.float32),
  tf.keras.metrics.Recall(name='recall', dtype=tf.float32),
  tf.keras.metrics.AUC(name='auc', dtype=tf.float32),
]

In [ ]:
def make_basic_block_layer(filter_num, blocks, stride=1):
  """
  Generate a block layer
  """
  res_block = tf.keras.Sequential()
  res_block.add(basic_block(filter_num, stride=stride))

  for _ in range(1, blocks):
    res_block.add(basic_block(filter_num, stride=1))

  return res_block

def basic_block(filter_num, stride=1):
  """
  Add a downsample layer depending on the  stride
  """
  model  = tf.keras.Sequential([
    layers.Conv2D(filters=filter_num, kernel_size=(3, 3), strides=stride, padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(filters=filter_num, kernel_size=(3, 3), strides=1, padding='same'),
    layers.BatchNormalization()
  ])

  if stride != 1:
    return tf.keras.Sequential([
      model,
      layers.Conv2D(filters=filter_num, kernel_size=(1, 1), strides=stride),
      layers.BatchNormalization(),
      layers.ReLU()
    ])
  else:
    return model

def make_bottleneck_layer(filter_num, blocks, stride=1):
  """
  Generate a bottleneck layer
  """
  res_block = tf.keras.Sequential()
  res_block.add(bottleneck_layer(filter_num, stride=stride))

  for _ in range(1, blocks):
    res_block.add(bottleneck_layer(filter_num, stride=1))

  return res_block

def bottleneck_layer(filter_num, stride=1):
  """
  Add a downsample layer
  """
  model  = tf.keras.Sequential([
    layers.Conv2D(filters=filter_num, kernel_size=(1, 1), strides=1, padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(filters=filter_num, kernel_size=(3, 3), strides=stride, padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(filters=filter_num * 4, kernel_size=(1, 1), strides=1, padding='same'),
    layers.BatchNormalization()
  ])

  return tf.keras.Sequential([
    model,
    layers.Conv2D(filters=filter_num * 4, kernel_size=(1, 1), strides=stride),
    layers.BatchNormalization(),
    layers.ReLU()
  ])

def resnet_18(final_activation):
  nodes = [2, 2, 2, 2]
  model = tf.keras.Sequential([
      layers.Conv2D(16, 3, padding='same', input_shape=(IMG_HEIGHT, IMG_WIDTH , 3), strides=2),
      layers.BatchNormalization(),
      layers.ReLU(),
      layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same'),
      make_basic_block_layer(filter_num=64, blocks=nodes[0]),
      make_basic_block_layer(filter_num=128, blocks=nodes[1], stride=2),
      make_basic_block_layer(filter_num=256, blocks=nodes[2], stride=2),
      make_basic_block_layer(filter_num=512, blocks=nodes[3], stride=2),
      layers.GlobalAveragePooling2D(),
      # layers.Dropout(0.1),
      layers.Dense(units=NUM_CLASSES, activation=final_activation, name='predictions')
  ])

  return model

def resnet_34(final_activation):
  nodes = [3, 4, 6, 3]
  model = tf.keras.Sequential([
    layers.Conv2D(16, 3, padding='same', input_shape=(IMG_HEIGHT, IMG_WIDTH , 3), strides=2),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same'),
    make_bottleneck_layer(filter_num=64, blocks=nodes[0]),
    make_bottleneck_layer(filter_num=128, blocks=nodes[1], stride=2),
    make_bottleneck_layer(filter_num=256, blocks=nodes[2], stride=2),
    make_bottleneck_layer(filter_num=512, blocks=nodes[3], stride=2),
    layers.GlobalAveragePooling2D(),
    # layers.Dropout(0.1),
    layers.Dense(units=NUM_CLASSES, activation=final_activation, name='predictions')
  ])

  return model

In [ ]:
def get_model_vgg_16(nodes=16, optimizer='adam', loss='binary_crossentropy', hidden_activation='linear', final_activation='softmax', metrics='accuracy'):
  """
  Return a VGG16 model - CF https://arxiv.org/pdf/1409.1556/
  """
  model = tf.keras.Sequential([
    layers.Conv2D(16, 3, padding='same', activation=hidden_activation, input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    layers.MaxPooling2D(),
    layers.Dropout(0.1),
    layers.Conv2D(32, 3, padding='same', activation=hidden_activation),
    layers.MaxPooling2D(),
    layers.Dropout(0.1),
    layers.Conv2D(64, 3, padding='same', activation=hidden_activation),
    layers.MaxPooling2D(),
    layers.Dropout(0.1),
    layers.Flatten(),
    layers.Dense(512, activation=final_activation),
    layers.Dense(units=NUM_CLASSES)
  ])

  model.compile(optimizer=optimizer,
                loss=loss,
                metrics=metrics)
  return model

# vgg16 or resnet18 or resnet34
MODEL_NAME = 'vgg16'

model = get_model_vgg_16(
        nodes=16,
        optimizer=OPTIMIZER,
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        hidden_activation=HIDDEN_ACTIVATION,
        final_activation=FINAL_ACTIVATION,
        metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy', dtype=tf.float32)]
      )

In [ ]:
def get_model_resnet_18(optimizer='adam', loss='binary_crossentropy', final_activation='softmax', metrics='accuracy'):
  """
  Return a resnet 18 model
  """
  model = resnet_18(final_activation=final_activation)
    model.compile(optimizer=optimizer,
                loss=loss,
                metrics=metrics)
    return model

    model = resnet_34(final_activation=final_activation)
    model.compile(optimizer=optimizer,
                loss=loss,
                metrics=metrics)
    return model

# vgg16 or resnet18 or resnet34
MODEL_NAME = 'resnet18'

model = get_model_resnet_18(
        optimizer=OPTIMIZER,
        loss=tf.keras.losses.CategoricalCrossentropy(),
        final_activation=FINAL_ACTIVATION,
        metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy', dtype=tf.float32)]
      )

In [ ]:
def get_model_resnet_34(optimizer='adam', loss='binary_crossentropy', final_activation='softmax', metrics='accuracy'):
  """
  Return a resnet 34 model
  """
  model = resnet_34(final_activation=final_activation)
  model.compile(optimizer=optimizer,
              loss=loss,
              metrics=metrics)
  return model

# vgg16 or resnet18 or resnet34
MODEL_NAME = 'resnet34'

model = get_model_resnet_34(
        optimizer=OPTIMIZER,
        loss=tf.keras.losses.CategoricalCrossentropy(),
        final_activation=FINAL_ACTIVATION,
        metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy', dtype=tf.float32)]
      )

In [ ]:
def save_model_h5(model=None, model_name='vgg16'):
  """
  Save a TF Model into h5 format
  """
  model.save('saved_model/{}/model.h5'.format(MODEL_NAME))
  print("Model saved successfully.")

def save_model_tf(model=None, model_name='vgg16'):
  """
  Save a TF Model into SavedModel format
  """
  # Reset metrics before saving so that loaded model has same state,
  # since metric states are not preserved by Model.save_weights
  model.reset_metrics()

  model.save('saved_model/{}/model'.format(MODEL_NAME), save_format='tf')
  print("Model saved successfully.")

In [ ]:
def get_callbacks():
  """
  Define the callbacks for the ML model
  """
  return [
    # tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10),
    tf.keras.callbacks.TensorBoard(os.path.join("logs/{}".format(MODEL_NAME), datetime.datetime.now().strftime("%Y%m%d-%H%M%S")), histogram_freq=1)
  ]

In [ ]:
def get_label(file_path):
  """
  Get the label of a file - Can be NORMAL, VIRUS OR BACTERIE
  """
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  return parts[-2] == CLASS_NAMES

In [ ]:
def decode_img(img):
  """
  Convert an image into a tensor with needed size
  """
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

In [ ]:
def process_path(file_path):
  """
  Process a file
  """
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [ ]:
def get_label_predicted(predictions):
  """
  Return the label for a given prediction array
  """
  preds = list(predictions)
  return CLASS_NAMES[preds.index(max(preds))]

In [ ]:
def generate_results(history, model, results, predictions):
  """
  Function that generate an html page and open it in the browser
  - history is the history for the model train steps
  - results is the metrics results get from the train step
  - predictions is the predictive result gotten for the test datas
  """
  folder_results_name = 'results/{}_{}'.format(MODEL_NAME, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  os.mkdir(folder_results_name, 0o755)

  # Generate the graph for accuracy, loss, val_accuracy, val_loss
  history_dict = history.history

  acc = history_dict['accuracy']
  val_acc = history_dict['val_accuracy']
  loss = history_dict['loss']
  val_loss = history_dict['val_loss']

  epochs_range = range(NB_EPOCHS)

  plt.figure(figsize=(8, 8))
  plt.subplot(1, 2, 1)
  plt.plot(epochs_range, acc, label='Training Accuracy')
  plt.plot(epochs_range, val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.title('Training and Validation Accuracy')

  plt.subplot(1, 2, 2)
  plt.plot(epochs_range, loss, label='Training Loss')
  plt.plot(epochs_range, val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.title('Training and Validation Loss')
  plt.yscale('log')

  # save figure
  plt.savefig('{}/accuracy_and_loss.png'.format(folder_results_name))
	# close matplotlib
  plt.close()

  with open('{}/results.txt'.format(folder_results_name), 'w+', newline='') as f:
    # Write metrics for testing purpose
    f.write('Normal, Virus or Bacteria {} trained model\n'.format(MODEL_NAME))
    if MODEL_NAME == 'vgg16':
      f.write('Optimizer : {}, Hidden activation : {}, Final activation : {}\n'.format(OPTIMIZER, HIDDEN_ACTIVATION, FINAL_ACTIVATION))
    elif MODEL_NAME == 'resnet18' or MODEL_NAME == 'resnet34':
      f.write('Optimizer : {}, Final activation : {}\n'.format(OPTIMIZER, FINAL_ACTIVATION))
    f.write('--------------------------------------------------------------\n')
    f.write('\n')
    f.write('METRICS :\n')
    for name, value in zip(model.metrics_names, results):
      f.write(f'{name} : {value}\n')
    f.write('\n')
    # Write predictions
    f.write('PREDICTIONS :\n')
    for predict in predictions:
      f.write('Predictions : {}, Result : {}\n'.format(str(predict), get_label_predicted(predict)))

In [ ]:
# To get the nb of steps and how many images we got
nb_normal_tr = len(os.listdir('{}/NORMAL'.format(TRAIN_DATA_PATH)))
nb_bacteria_tr = len(os.listdir('{}/BACTERIA'.format(TRAIN_DATA_PATH)))
nb_virus_tr = len(os.listdir('{}/VIRUS'.format(TRAIN_DATA_PATH)))
nb_normal_val = len(os.listdir('{}/NORMAL'.format(VAL_DATA_PATH)))
nb_bacteria_val = len(os.listdir('{}/BACTERIA'.format(VAL_DATA_PATH)))
nb_virus_val = len(os.listdir('{}/VIRUS'.format(VAL_DATA_PATH)))
total_train = nb_normal_tr + nb_bacteria_tr + nb_virus_tr
total_val = nb_normal_val + nb_bacteria_val + nb_virus_val

# Our datas generators
train_image_generator = ImageDataGenerator(
  rescale=1./255,
  rotation_range=45,
  width_shift_range=.15,
  height_shift_range=.15,
  horizontal_flip=True,
  zoom_range=0.5
) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data
test_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

train_data_gen = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
  directory=TRAIN_DATA_PATH,
  shuffle=True,
  target_size=(IMG_HEIGHT, IMG_WIDTH),
  class_mode='categorical'
)
val_data_gen = validation_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
  directory=VAL_DATA_PATH,
  target_size=(IMG_HEIGHT, IMG_WIDTH),
  class_mode='categorical'
)
test_data_gen = test_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
  directory=TEST_DATA_PATH,
  target_size=(IMG_HEIGHT, IMG_WIDTH),
  class_mode='categorical'
)

# Train the model
history = model.fit(
  train_data_gen,
  callbacks=get_callbacks(),
  steps_per_epoch=total_train // BATCH_SIZE,
  epochs=NB_EPOCHS,
  validation_data=val_data_gen,
  validation_steps=total_val // BATCH_SIZE
)

model.summary()

# Use a testing model to display metrics
testing_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
testing_model.compile(
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
  optimizer='adam',
  metrics=METRICS
)

# Evalutate the model with test datas
results = testing_model.evaluate(test_data_gen, verbose=0)

# Predict the test datas
predictions = testing_model.predict(test_data_gen)

generate_results(history, testing_model, results, predictions)
# save_model_h5(testing_model, 'resnet_18')